In [1]:
pip install asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
rows = cw.query_produrements()

urls = []
for row in rows:
    url = 'https://{base}/portaltransparencia-api/api/licitacoes/arquivos?entidade=1&exercicio={year}&tipoLicitacao={type}&licitacao={num}'.format(
        base=cw._url,
        year=row[2],
        type=row[3],
        num=row[4]
    )
    urls.append(url)

Scraping.crawlers.crawler_oxy


ModuleNotFoundError: No module named 'ironpdf'

In [ ]:
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

responses = []

async def fetch(client, url):
    async with client.get(url) as resp:
        assert resp.status == 200
        html = await resp.json()
        responses.append(html)


async def main():
    async with aiohttp.ClientSession() as client:
        await asyncio.gather(*[
            asyncio.ensure_future(fetch(client, url))
            for url in urls
        ])


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
pip install nest-asyncio


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
i=0
to_insert = []
for response in responses:
    i += 1
    for item in response:
        id = str(item['licitacao']) + '/' + str(item['exercicio']) + '/' + str(item['tipoLicitacao'])
        to_insert.append((cw._id_ibge, id, item['arquivoCompras']['idArquivo'], os.path.join("/" + str(cw._id_ibge), id, str(item['arquivoCompras']['idArquivo']) + ".pdf"), item['arquivoCompras']['nomeArquivo'], item['arquivoCompras']['data']))

In [ ]:
import sqlite3

In [ ]:
insert = "INSERT OR IGNORE INTO FILES (ibge, procurement_id, file_id, path, OCR_RAW, name, date) VALUES (?, ?, ?, ?, '', ?, ?)"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_insert)
con.commit()
con.close()

In [ ]:
pip install aiofile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')

Scraping.crawlers.crawler_oxy


In [ ]:
rows = cw.query_files(5)
rows

[(4119905, '1/2015/6', 6429),
 (4119905, '1/2015/7', 6715),
 (4119905, '1/2015/9', 6729),
 (4119905, '1/2016/6', 6099),
 (4119905, '1/2016/7', 6139)]

In [ ]:
import os
rows = cw.query_files()

i = 1
to_update = []
for row in rows:   
    file = os.path.join("", "Data", row[0].__str__(), row[1].replace("/", "-"))
    to_update.append((path, row[0], row[1], row[2]))

In [ ]:
insert = "UPDATE FILES set path = ? where ibge = ? and procurement_id = ? and file_id = ?"
con = sqlite3.connect("scraped.db", timeout=50)
cursor = con.cursor()
cursor.executemany(insert, to_update)
con.commit()
con.close()

In [ ]:
#Validar pastas
pastas = cw.query_folders()
for pasta in pastas:
    if not os.path.exists(pasta[0]):
        os.makedirs(pasta[0]) 

In [ ]:
files = cw.query_files(3)
files2 = [('https://{base}/portaltransparencia-api/api/files/arquivo/{file_id}'.format(
                base=cw._url,
                file_id=x[2]
            ), ) + x for x in files]
files2

[('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6429',
  4119905,
  '1/2015/6',
  6429,
  'Data\\4119905\\1-2015-6',
  '6429.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6715',
  4119905,
  '1/2015/7',
  6715,
  'Data\\4119905\\1-2015-7',
  '6715.pdf'),
 ('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729',
  4119905,
  '1/2015/9',
  6729,
  'Data\\4119905\\1-2015-9',
  '6729.pdf')]

In [ ]:
for file in files2:
    print(file)

('https://cmpg.oxy.elotech.com.br/portaltransparencia-api/api/files/arquivo/6729', 4119905, '1/2015/9', 6729, 'Data\\4119905\\1-2015-9', '6729.pdf')


In [ ]:
pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Check if file exists
files3 = [x for x in files2 if not os.path.isfile(os.path.join(x[4], x[5]))]

In [ ]:
#baixar e salvar cada arquivo

import os
import asyncio
import aiohttp  # pip install aiohttp
import aiofile  # pip install aiofile
import nest_asyncio

nest_asyncio.apply()

def download_files_from_report(files):

    #sema = asyncio.BoundedSemaphore(5)

    async def fetch_file(session, file):
        
        url = file[0]
        folder = file[4]
        fname = file[5]

        #async with sema:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

        async with aiofile.async_open(
            os.path.join(folder, fname), "wb"
        ) as outfile:
            await outfile.write(data)

    async def main():
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_file(session, file) for file in files]
            await asyncio.gather(*tasks)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    #loop.close()

download_files_from_report(files3)

In [ ]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')
try:
    cw.get_all_files()
except:
    pass

Scraping.crawlers.crawler_oxy
there is 2483 files in the database
there is 1827 files to be downloaded
the first file id is: 6139.
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded
there is 2483 files in the database
there is 0 files to be downloaded


IndexError: list index out of range

In [2]:
pip install IronPdf --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from Scraping import *

cw = crawlers.CrawlerMaster(1, 'cmpg.oxy.elotech.com.br', 4119905, 'Ponta Grossa/PR')

import nest_asyncio
nest_asyncio.apply()

cw.do_ocr(2)

## __init__ crawlers load: oxy
## Starting Scraping.crawlers.crawler_oxy ##
## Starting Scraping.crawlers.crawler_base ##
## __init__ crawlers load: base
## __init__ crawlers load: master
## Starting Scraping.crawlers.crawler_master ##
[(4119905, '1/2015/6', 6429, 'Files\\4119905\\1-2015-6', '6429.pdf'), (4119905, '1/2015/7', 6715, 'Files\\4119905\\1-2015-7', '6715.pdf')]


FileNotFoundError: [Errno 2] No such file or directory: 'Files\\4119905\\1-2015-6\\6429.pdf\\temp_image_0.png'